# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data/cities.csv')
weather_df.head()

,City_Name,country,Latitude,Longitude,Temperature/F,Humidity/%,Cloudiness/%,Wind_Speed/mph
0,hilo,US,19.73,-155.09,78.80,61,40,11.41
1,arraial do cabo,BR,-22.97,-42.03,77.00,73,40,8.05
2,vaini,TO,-21.20,-175.20,82.40,65,20,18.34
3,coahuayana,MX,18.73,-103.68,89.60,40,40,3.40
4,thohoyandou,ZA,-22.95,30.48,66.06,87,100,4.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [14]:
locations = weather_df[['Latitude', 'Longitude']]
weight = weather_df['Humidity/%']

figure_layout={
    'width':'200px',
    'height':'300px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
figure = gmaps.figure(figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=weight, max_intensity=max(weight)+20, point_radius=2, dissipating=False)
figure.add_layer(heat_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_df.loc[(weather_df['Temperature/F']>70)&(weather_df['Temperature/F']<85)\
                                 &(weather_df['Wind_Speed/mph']<10)&(weather_df['Cloudiness/%']== 0)]

In [6]:
ideal_weather_df.head()

,City_Name,country,Latitude,Longitude,Temperature/F,Humidity/%,Cloudiness/%,Wind_Speed/mph
84,sao filipe,CV,14.90,-24.50,70.39,75,0,8.88
137,alibag,IN,18.64,72.88,71.60,73,0,2.24
152,ngunguru,NZ,-35.62,174.50,81.00,64,0,5.01
284,necochea,AR,-38.55,-58.74,80.01,59,0,3.00
296,hukuntsi,BW,-24.00,21.75,74.89,27,0,5.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame()
hotel_df['city'] = ideal_weather_df['City_Name']
hotel_df['country'] = ideal_weather_df['country']
hotel_df['Latitude'] = ideal_weather_df['Latitude']
hotel_df['Longitude'] = ideal_weather_df['Longitude']
hotel_df['Hotel Name'] = ''

In [8]:
for index, row in ideal_weather_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    target_coordinates = f'{row.Latitude},{row.Longitude}'
    params = {
        "location":target_coordinates,
#         "rankby": 'distance',
        "type": 'hotel',
        "radius": 5000,
        "key": g_key
    }
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


In [16]:
hotel_df

,city,country,Latitude,Longitude,Hotel Name
84,sao filipe,CV,14.90,-24.50,Sao Filipe
137,alibag,IN,18.64,72.88,Alibag
152,ngunguru,NZ,-35.62,174.50,Ngunguru
284,necochea,AR,-38.55,-58.74,Necochea
296,hukuntsi,BW,-24.00,21.75,Hukuntsi
449,kawhia,NZ,-38.07,174.82,Kawhia


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure_hotel = figure
figure_hotel.add_layer(markers)
figure_hotel
# Display Map

Figure(layout=FigureLayout(height='420px'))